## Extended Kalman filter for Nomoto model

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import sympy as sp

import src.visualization.book_format as book_format
book_format.set_style()
from src.substitute_dynamic_symbols import lambdify
from sympy import Matrix
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from IPython.display import display, Math, Latex
from src.substitute_dynamic_symbols import run, lambdify
from sympy.physics.vector.printing import vpprint, vlatex
from src.data import mdl
from src.kalman_filter import ssa, filter_yaw

## Load test

In [ ]:
id=22773
df, units, meta_data = mdl.load(id=id)

In [ ]:
from src.visualization.plot import track_plot
fig,ax=plt.subplots()
fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

# First order Nomoto model for ship maneuvering dynamics
The first order Nomoto model can be written as:

In [ ]:
K, delta, T_1, T_2 = sp.symbols('K delta T_1 T_2')
r = dynamicsymbols('r')

eq_nomoto = sp.Eq(-K*delta,
                 r + T_1*r.diff() + T_2*r.diff().diff())
Math(vlatex(eq_nomoto))

where $r$ is yaw rate with its time derivatives and $\delta$ is the rudder angle. $K$, $T_{1}$
 and $T_{1}$ are the coefficients describing the hydrodynamics of the ship.
 
For slow manoeuvres this equation can be further simplified by removing the $\ddot{r}$ term:

In [ ]:
eq_nomoto_simple = eq_nomoto.subs(r.diff().diff(),0)
Math(vlatex(eq_nomoto_simple))

In [ ]:
sp.solve(eq_nomoto_simple,r, dict=True)[0][r]

In [ ]:
sp.solve(eq_nomoto_simple,r.diff(), dict=True)[0][r.diff()]

In [ ]:
psi = sp.symbols('psi')
x = Matrix([psi,r,r.diff()])
x

In [ ]:
u = Matrix([delta])

In [ ]:
h = sp.symbols('h')
eqs = [
0,
sp.solve(eq_nomoto_simple,r, dict=True)[0][r],
sp.solve(eq_nomoto_simple,r.diff(), dict=True)[0][r.diff()]
]

A = Matrix(eqs).jacobian(x)
A[0,1] = 1
A

In [ ]:
Phi = sp.eye(3) + A*h
Phi

In [ ]:
Ad = Phi

In [ ]:
B = Matrix(eqs).jacobian(u)
B

In [ ]:
Bd = B*h
Bd

In [ ]:
sp.UnevaluatedExpr(Ad)*sp.UnevaluatedExpr(x) + sp.UnevaluatedExpr(Bd)*sp.UnevaluatedExpr(u)

In [ ]:
lambda_Ad = lambdify(Ad)
lambda_Bd = lambdify(Bd)

In [ ]:
T_1_ = 1.8962353076056344
K_ = 0.17950970687951323
h_ = 0.01

Ad_=lambda_Ad(T_1=T_1_, h=h_)
Bd_=lambda_Bd(K=K_, T_1=T_1_, h=h_)

In [ ]:
np.random.seed(42)
N_ = 1000
t_ = np.arange(0,N_*h_,h_)

us = np.deg2rad(np.concatenate((-10*np.ones(int(N_/4)),
                                10*np.ones(int(N_/4)),
                                -10*np.ones(int(N_/4)),
                                10*np.ones(int(N_/4)))))

simdata = []
x_=np.array([[0,0,0]]).T
for u_ in us:
    
    x_ = Ad_@x_ + Bd_*u_

    simdata.append(x_.flatten())
    
simdata = np.array(simdata)
df = pd.DataFrame(simdata, columns=["psi","r", "r1d"], index=t_)
measurement_noise = np.deg2rad(1)
df['epsilon'] = measurement_noise*np.random.normal(size=N_)

df['psi_measure'] = df['psi'] + df['epsilon']
df['psi_deg'] = np.rad2deg(df['psi'])
df['psi_measure_deg'] = np.rad2deg(df['psi_measure'])
df['delta'] = us
df['delta_deg'] = np.rad2deg(df['delta'])


In [ ]:
fig,ax=plt.subplots()
df.plot(y='psi_deg', ax=ax)
df.plot(y='psi_measure_deg', ax=ax, zorder=-1)
df.plot(y='delta_deg', ax=ax, zorder=-1)

df.plot(y='r')
df.plot(y='r1d')


In [ ]:
x0=np.deg2rad(np.array([[0,0,0]]).T)
P_prd = np.diag(np.deg2rad([1, 0.1, 0.1]))

Qd = np.deg2rad(np.diag([0, 0.1, 0.1]))

Rd = np.deg2rad(10)

ys = df['psi_measure'].values

E_ = np.array(
    [[0,0,0], [1,0,0], [1,0,0]],
)

C_ = np.array([[1, 0, 0]])

Cd_ = C_
Ed_ = h_ * E_

time_steps = filter_yaw(x0=x0, P_prd=P_prd,h_m=h_,h=h_, us=us, ys=ys, Ad=Ad_, Bd=Bd_, Cd=Cd_, Ed=Ed_, Qd=Qd, Rd=Rd)
x_hats = np.array([time_step["x_hat"] for time_step in time_steps]).T
time = np.array([time_step["time"] for time_step in time_steps]).T
Ks = np.array([time_step["K"] for time_step in time_steps]).T


In [ ]:
n=len(Ad_)
fig,axes=plt.subplots(nrows=n)

keys = ['psi','r','r1d']
for i,key in enumerate(keys):
    
    ax=axes[i]
    df.plot(y=key, ax=ax, label="True")
    if key=='psi':
        df.plot(y='psi_measure', ax=ax, label="Measured", zorder=-1)
    
    ax.plot(time, x_hats[i, :], "--", label="kalman")
    ax.set_ylabel(key)
    ax.legend()

In [ ]:
fig,ax=plt.subplots()
for i,key in enumerate(keys):
    ax.plot(time,Ks[i,:],label=key)
ax.set_title('Kalman gains')
ax.legend()
